# Deploy Trained Smolvla Policy

<img src="./media/rollout3.gif" width="480" height="360">

Deploy trained policy in simulation.

In [1]:
!pip install transformers==4.50.3
!pip install num2words
!pip install accelerate
!pip install safetensors>=0.4.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## Step 3. Deploy

In [2]:
# Dataset imports - moved from lerobot.common.datasets
from lerobot.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
from lerobot.datasets.utils import write_json, serialize_dict, dataset_to_policy_features
from lerobot.datasets.factory import resolve_delta_timestamps

# SmolVLA policy imports - moved from lerobot.common.policies
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy

# Config types - moved from lerobot.configs
from lerobot.configs.types import FeatureType

# Standard imports remain the same
import numpy as np
import torch
from PIL import Image
import torchvision


/home/dragon/anaconda3/envs/mujoco_vla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda'

In [4]:
try:
    dataset_metadata = LeRobotDatasetMetadata("omy_pnp_language", root='./demo_data_language')
except:
    dataset_metadata = LeRobotDatasetMetadata("omy_pnp_language", root='./omy_pnp_language')
features = dataset_to_policy_features(dataset_metadata.features)
output_features = {key: ft for key, ft in features.items() if ft.type is FeatureType.ACTION}
input_features = {key: ft for key, ft in features.items() if key not in output_features}
# Policies are initialized with a configuration class, in this case `DiffusionConfig`. For this example,
# we'll just use the defaults and so no arguments other than input/output features need to be passed.
# Temporal ensemble to make smoother trajectory predictions
cfg = SmolVLAConfig(input_features=input_features, output_features=output_features, chunk_size= 5, n_action_steps=5)
delta_timestamps = resolve_delta_timestamps(cfg, dataset_metadata)

In [5]:
# 加载本地训练的policy（专门用于移除红块任务）
# local_policy = SmolVLAPolicy.from_pretrained("DragonHu/smolvla_UR52_empty_plat", dataset_stats=dataset_metadata.stats)
local_policy = SmolVLAPolicy.from_pretrained('./ckpt/smolvla_omy/checkpoints/last/pretrained_model', dataset_stats=dataset_metadata.stats)
local_policy.to(device)
local_policy.eval()  # 设置为评估模式
print("✅ 本地policy加载完成 - 专门用于移除红块任务")

# 加载云端policy（专门用于放置蓝杯任务）
try:
    cloud_policy = SmolVLAPolicy.from_pretrained("Jeongeun/omy_pnp_smolvla", config=cfg, dataset_stats=dataset_metadata.stats)
    cloud_policy.to(device)
    cloud_policy.eval()  # 设置为评估模式
    print("✅ 云端policy加载完成 - 专门用于放置蓝杯任务")
except Exception as e:
    print(f"❌ 加载云端policy失败: {e}")
    cloud_policy = None

print("🎯 Policy配置完成：")
print("   - 第一阶段（移除红块）：使用 local_policy")
print("   - 第二阶段（放置蓝杯）：使用 cloud_policy")

Reducing the number of VLM layers to 16 ...
Loading weights from local directory
✅ 本地policy加载完成 - 专门用于移除红块任务
Reducing the number of VLM layers to 16 ...
✅ 云端policy加载完成 - 专门用于放置蓝杯任务
🎯 Policy配置完成：
   - 第一阶段（移除红块）：使用 local_policy
   - 第二阶段（放置蓝杯）：使用 cloud_policy


In [6]:
from mujoco_env.y_env2_removeBlock_Pnp_mug_Test import SimpleEnvRemoveBlock_pnp_mug
xml_path = './asset/scene_remove_block.xml'
PnPEnv = SimpleEnvRemoveBlock_pnp_mug(xml_path, action_type='joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[31] n_qvel:[28] n_qacc:[28] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[22]
 [0/22] [world] mass:[0.00]kg
 [1/22] [front_object_table] mass:[1.00]kg
 [2/22] [camera] mass:[0.00]kg
 [3/22] [camera2] mass:[0.00]kg
 [4/22] [camera3] mass:[0.00]kg
 [5/22] [link1] mass:[2.06]kg
 [6/22] [link2] mass:[3.68]kg
 [7/22] [link3] mass:[2.39]kg
 [8/22] [link4] mass:[1.40]kg
 [9/22] [link5] mass:[1.40]kg
 [10/22] [link6] mass:[0.65]kg
 [11/22] [camera_center] mass:[0.00]kg
 [12/22] [tcp_link] mass:[0.32]kg
 [13/22] [rh_p12_rn_r1] mass:[0.07]kg
 [14/22] [rh_p12_rn_r2] mass:[0.02]kg
 [15/22] [rh_p12_rn_l1] mass:[0.07]kg
 [16/22] [rh_p12_rn_l2] mass:[0.02]kg
 [17/22] [body_obj_plate_11] mass:[0.00]kg
 [18/22] [object_plate_11] mass:[0.10]kg
 [19/22] [body_obj_mug_6] mass:[0.00]kg
 [20/22] [object_mug_6] mass:[0.08]kg
 [21/22] [body_obj_block_red] mass:[0.30]kg
body_total_mass:

In [7]:
from torchvision import transforms
# Approach 1: Using torchvision.transforms
def get_default_transform(image_size: int = 224):
    """
    Returns a torchvision transform that:
     Converts to a FloatTensor and scales pixel values [0,255] -> [0.0,1.0]
    """
    return transforms.Compose([
        transforms.ToTensor(),  # PIL [0–255] -> FloatTensor [0.0–1.0], shape C×H×W
    ])

In [8]:
# 双任务测试配置参数
# 可以根据需要修改这些参数

# 测试轮数设置
TEST_ROUNDS = 20  # 默认测试10轮，可以修改为任意数字

# 超时设置（秒）- 每个任务独立计时
TASK_TIMEOUT_SECONDS = 90  # 默认每个任务2分钟超时，可以修改

# 设置环境的超时时间
PnPEnv.task_timeout = TASK_TIMEOUT_SECONDS

print(f"双任务测试配置:")
print(f"- 测试轮数: {TEST_ROUNDS}")
print(f"- 每个任务超时时间: {TASK_TIMEOUT_SECONDS} 秒")
print(f"- 每轮包含任务: 1) 移除红块 2) 放置蓝杯")
print(f"- 预计最大测试时间: {TEST_ROUNDS * TASK_TIMEOUT_SECONDS * 2 / 60:.1f} 分钟")
print("准备开始双任务测试...")


双任务测试配置:
- 测试轮数: 20
- 每个任务超时时间: 90 秒
- 每轮包含任务: 1) 移除红块 2) 放置蓝杯
- 预计最大测试时间: 60.0 分钟
准备开始双任务测试...


In [9]:
# 开始双任务测试循环
step = 0
IMG_TRANSFORM = get_default_transform()

# 开始测试循环
PnPEnv.start_testing(max_rounds=TEST_ROUNDS)
local_policy.reset()
local_policy.eval()
if cloud_policy is not None:
    cloud_policy.reset()
    cloud_policy.eval()

print(f"🚀 开始进行 {TEST_ROUNDS} 轮双任务SmolVLA成功率测试...")
print("📋 Policy配置确认：")
print(f"   - 第一阶段（移除红块）：{'✅ 本地policy' if local_policy is not None else '❌ 未加载'}")
print(f"   - 第二阶段（放置蓝杯）：{'✅ 云端policy' if cloud_policy is not None else '❌ 未加载，将使用本地policy'}")
print("=" * 50)

while PnPEnv.env.is_viewer_alive() and PnPEnv.is_testing:
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # 检查任务超时
        if PnPEnv.check_task_timeout():
            current_task = PnPEnv.current_task
            if current_task == 'remove_red_block':
                print(f"任务1(移除红块)超时！已运行 {PnPEnv.task_timeout} 秒")
                # 任务1失败，直接结束本轮测试
                all_tests_completed = PnPEnv.handle_dual_task_completion(
                    task1_success=False, task2_success=False, 
                    task1_timeout=True, task2_timeout=False
                )
            else:  # place_blue_mug
                print(f"任务2(放置蓝杯)超时！已运行 {PnPEnv.task_timeout} 秒")
                # 任务1成功但任务2超时
                all_tests_completed = PnPEnv.handle_dual_task_completion(
                    task1_success=True, task2_success=False, 
                    task1_timeout=False, task2_timeout=True
                )
            
            if all_tests_completed:
                break
            # 重置policies
            local_policy.reset()
            if cloud_policy is not None:
                cloud_policy.reset()
            step = 0
            continue
        
        # 检查当前任务是否完成
        success = PnPEnv.check_success()
        
        # 如果红块移除任务完成，切换到蓝杯放置任务
        if success and PnPEnv.current_task == 'remove_red_block':
            print('🎉 红块移除任务完成！')
            if PnPEnv.switch_to_next_task():
                # 重置云端policy状态，准备执行蓝杯放置任务
                if cloud_policy is not None:
                    cloud_policy.reset()
                    PnPEnv.policy_switch_count += 1
                    print(f"🔄 准备使用云端policy执行蓝杯放置任务 (切换次数: {PnPEnv.policy_switch_count})")
                    print("📝 确保使用正确的指令: 'Place the blue mug on the plate'")
                else:
                    print("⚠️ 警告：云端policy未加载，将使用本地policy执行蓝杯放置任务")
        
        # 检查所有任务是否都完成
        elif PnPEnv.check_all_tasks_completed():
            print('🎊 双任务全部完成！')
            # 双任务成功完成
            all_tests_completed = PnPEnv.handle_dual_task_completion(
                task1_success=True, task2_success=True, 
                task1_timeout=False, task2_timeout=False
            )
            
            if all_tests_completed:
                break
            
            # 重置所有policy状态，准备下一轮测试
            local_policy.reset()
            if cloud_policy is not None:
                cloud_policy.reset()
            step = 0
            continue
            
        # Get the current state of the environment
        state = PnPEnv.get_joint_state()[:6]
        # Get the current image from the environment
        image, wirst_image = PnPEnv.grab_image()
        image = Image.fromarray(image)
        image = image.resize((256, 256))
        image = IMG_TRANSFORM(image)
        wrist_image = Image.fromarray(wirst_image)
        wrist_image = wrist_image.resize((256, 256))
        wrist_image = IMG_TRANSFORM(wrist_image)
        
        # 根据当前任务直接选择对应的policy和指令
        if PnPEnv.current_task == 'place_blue_mug':
            # 第二阶段：使用云端policy执行蓝杯放置任务
            instruction = "Place the blue mug on the plate"
            selected_policy = cloud_policy if cloud_policy is not None else local_policy
            policy_name = "云端policy" if cloud_policy is not None else "本地policy(备用)"
        else:
            # 第一阶段：使用本地policy执行红块移除任务
            instruction = PnPEnv.instruction
            selected_policy = local_policy
            policy_name = "本地policy"
            
        data = {
            'observation.state': torch.tensor([state]).to(device),
            'observation.image': image.unsqueeze(0).to(device),
            'observation.wrist_image': wrist_image.unsqueeze(0).to(device),
            'task': [instruction],
        }
        
        # 直接使用对应任务的policy选择action
        action = selected_policy.select_action(data)
        action = action[0,:7].cpu().detach().numpy()
        
        # Take a step in the environment
        _ = PnPEnv.step(action)
        
        # 渲染环境，显示测试进度
        progress = PnPEnv.get_dual_task_progress()
        if progress:
            PnPEnv.render(idx=progress['current_round'])
        else:
            PnPEnv.render()
        
        step += 1
        
        # 显示测试进度信息 - 增加显示频率
        if step % 20 == 0 and progress:  # 每20步显示一次进度
            current_task_name = "移除红块" if progress['current_task'] == 'remove_red_block' else "放置蓝杯"
            print(f"测试进度: {progress['current_round']}/{progress['total_rounds']}, "
                  f"当前任务: {current_task_name}, "
                  f"任务用时: {progress['task_elapsed_time']:.1f}s, "
                  f"总用时: {progress['total_elapsed_time']:.1f}s")
            print(f"  任务状态 - 红块移除: {'✅' if progress['task1_completed'] else '⏳'}, "
                  f"蓝杯放置: {'✅' if progress['task2_completed'] else '⏳'}")
            print(f"  使用policy: {policy_name}")

开始双任务测试循环，总共将进行 20 轮测试
每轮测试包含: 1) 移除红块 2) 放置蓝杯
DONE INITIALIZATION
🚀 开始进行 20 轮双任务SmolVLA成功率测试...
📋 Policy配置确认：
   - 第一阶段（移除红块）：✅ 本地policy
   - 第二阶段（放置蓝杯）：✅ 云端policy


/tmp/ipykernel_40507/2356310384.py:108: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  'observation.state': torch.tensor([state]).to(device),


测试进度: 1/20, 当前任务: 移除红块, 任务用时: 1.8s, 总用时: 1.8s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 3.3s, 总用时: 3.3s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 4.9s, 总用时: 4.9s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 6.4s, 总用时: 6.4s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 8.0s, 总用时: 8.0s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 9.5s, 总用时: 9.5s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 11.1s, 总用时: 11.1s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 12.7s, 总用时: 12.7s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 14.3s, 总用时: 14.3s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 15.8s, 总用时: 15.8s
  任务状态 - 红块移除: ⏳, 蓝杯放置: ⏳
  使用policy: 本地policy
测试进度: 1/20, 当前任务: 移除红块, 任务用时: 17.3s, 总用时: 17.3s
  任务状态 - 红块移除:

In [10]:
print("双任务测试完成！")
# 如果测试被中断，也显示当前结果
if hasattr(PnPEnv, 'test_stats') and PnPEnv.test_stats['total_tests'] > 0:
    final_results = PnPEnv.print_dual_task_results()
    print("\n📋 最终双任务测试结果摘要:")
    print(f"🎯 双任务成功率: {final_results['success_rate']:.2f}%")
    print(f"📊 总测试次数: {final_results['total_tests']}")
    print(f"✅ 双任务成功: {final_results['successful_tests']}")
    print(f"❌ 测试失败: {final_results['failed_tests']}")
    print(f"⏰ 超时失败: {final_results['timeout_tests']}")
    print(f"🔴 任务1失败: {final_results['task1_failed']}")
    print(f"🔵 任务2失败: {final_results['task2_failed']}")
    
    # 显示时间摘要
    if final_results['total_success_times']:
        total_times = final_results['total_success_times']
        avg_total = sum(total_times) / len(total_times)
        print(f"⏱️ 平均双任务完成时间: {avg_total:.1f}秒")
        print(f"⚡ 最快双任务完成时间: {min(total_times):.1f}秒")
        print(f"🐌 最慢双任务完成时间: {max(total_times):.1f}秒")
    
    if final_results['task1_success_times']:
        task1_times = final_results['task1_success_times']
        avg_task1 = sum(task1_times) / len(task1_times)
        print(f"🔴 平均任务1完成时间: {avg_task1:.1f}秒")
    
    if final_results['task2_success_times']:
        task2_times = final_results['task2_success_times']
        avg_task2 = sum(task2_times) / len(task2_times)
        print(f"🔵 平均任务2完成时间: {avg_task2:.1f}秒")
    
    print("🔚 MuJoCo窗口已自动关闭，双任务测试结束")


双任务测试完成！

双任务测试结果统计
总测试轮数: 20
双任务成功: 4 轮
测试失败: 16 轮
超时失败: 16 轮
双任务成功率: 20.00%

任务失败分析:
  任务1(移除红块)成功: 19 轮 / 失败: 1 轮 (成功率: 95.00%)
  任务2(放置蓝杯)成功: 4 轮 / 失败: 15 轮 (尝试: 19 轮, 全局成功率: 20.00%, 尝试成功率: 21.05%)
  任务2未尝试: 1 轮 (因任务1未完成)

任务1时间统计:
  平均用时: 29.9秒
  最短用时: 11.3秒
  最长用时: 67.0秒
  所有时间: ['43.1s', '13.6s', '20.8s', '14.2s', '19.4s', '33.2s', '22.3s', '52.2s', '22.7s', '61.8s', '17.9s', '42.5s', '37.0s', '28.6s', '67.0s', '12.4s', '11.3s', '23.8s', '23.8s']

任务2时间统计:
  平均用时: 12.7秒
  最短用时: 11.5秒
  最长用时: 13.9秒
  所有时间: ['12.2s', '13.9s', '13.0s', '11.5s']

双任务总体时间统计:
  平均总用时: 46.1秒
  最短总用时: 28.1秒
  最长总用时: 67.8秒
  所有总时间: ['28.1s', '67.8s', '51.6s', '37.0s']

📋 最终双任务测试结果摘要:
🎯 双任务成功率: 20.00%
📊 总测试次数: 20
✅ 双任务成功: 4
❌ 测试失败: 16
⏰ 超时失败: 16
🔴 任务1失败: 1
🔵 任务2失败: 15
⏱️ 平均双任务完成时间: 46.1秒
⚡ 最快双任务完成时间: 28.1秒
🐌 最慢双任务完成时间: 67.8秒
🔴 平均任务1完成时间: 29.9秒
🔵 平均任务2完成时间: 12.7秒
🔚 MuJoCo窗口已自动关闭，双任务测试结束


In [11]:
# 详细双任务测试结果分析
if hasattr(PnPEnv, 'test_stats') and PnPEnv.test_stats['total_tests'] > 0:
    results = PnPEnv.test_stats
    total = results['total_tests']
    success = results['successful_tests']
    failed = results['failed_tests']
    timeout = results['timeout_tests']
    task1_failed = results['task1_failed']
    task2_failed = results['task2_failed']

    task1_times = results.get('task1_success_times', [])
    task2_times = results.get('task2_success_times', [])
    total_times = results.get('total_success_times', [])

    task1_completed = results.get('task1_completed', total - task1_failed)
    task2_success = results.get('task2_success', len(task2_times))
    task2_attempts = results.get('task2_attempts', task2_success + task2_failed)

    print('\n' + '='*70)
    print('SmolVLA 双任务成功率测试 - 详细分析报告')
    print('='*70)

    # 基本统计
    print('📊 基本统计:')
    print(f'   总测试轮数: {total}')
    print(f'   双任务成功: {success} 轮')
    print(f'   测试失败: {failed} 轮')
    print(f'   超时失败: {timeout} 轮')

    # 成功率计算
    success_rate = (success / total * 100) if total > 0 else 0
    print(f'\n🎯 双任务成功率: {success_rate:.2f}%')

    # 任务分解分析
    print(f'\n🔍 任务失败分解分析:')
    task1_success_rate = (task1_completed / total * 100) if total > 0 else 0
    task2_success_rate_total = (task2_success / total * 100) if total > 0 else 0
    task2_success_rate_attempts = (task2_success / task2_attempts * 100) if task2_attempts > 0 else 0
    print(f'   任务1(移除红块)成功率: {task1_success_rate:.2f}% ({task1_completed}/{total})')
    print(f'   任务2(放置蓝杯)成功率(全局): {task2_success_rate_total:.2f}% ({task2_success}/{total})')
    if task2_attempts > 0:
        print(f'   任务2(放置蓝杯)成功率(尝试): {task2_success_rate_attempts:.2f}% ({task2_success}/{task2_attempts})')
        if task2_attempts < total:
            print(f'   任务2未尝试: {total - task2_attempts} 轮 (因任务1失败)')
    print(f'   任务1失败: {task1_failed} 轮')
    print(f'   任务2失败: {task2_failed} 轮')

    # 时间分析
    if task1_times:
        avg_task1 = sum(task1_times) / len(task1_times)
        formatted_task1_times = [f"{t:.1f}s" for t in task1_times]
        print(f'\n⏱️ 任务1时间分析:')
        print(f'   平均完成时间: {avg_task1:.1f}秒')
        print(f'   最快完成时间: {min(task1_times):.1f}秒')
        print(f'   最慢完成时间: {max(task1_times):.1f}秒')
        print(f'   所有完成时间: {formatted_task1_times}')

        # 任务1速度评估
        if avg_task1 <= 30:
            print('   ⚡ 任务1速度评估: 非常快速')
        elif avg_task1 <= 60:
            print('   🚀 任务1速度评估: 较快')
        elif avg_task1 <= 90:
            print('   🐌 任务1速度评估: 一般')
        else:
            print('   🐢 任务1速度评估: 较慢')

    if task2_times:
        avg_task2 = sum(task2_times) / len(task2_times)
        formatted_task2_times = [f"{t:.1f}s" for t in task2_times]
        print(f'\n⏱️ 任务2时间分析:')
        print(f'   平均完成时间: {avg_task2:.1f}秒')
        print(f'   最快完成时间: {min(task2_times):.1f}秒')
        print(f'   最慢完成时间: {max(task2_times):.1f}秒')
        print(f'   所有完成时间: {formatted_task2_times}')

        # 任务2速度评估
        if avg_task2 <= 30:
            print('   ⚡ 任务2速度评估: 非常快速')
        elif avg_task2 <= 60:
            print('   🚀 任务2速度评估: 较快')
        elif avg_task2 <= 90:
            print('   🐌 任务2速度评估: 一般')
        else:
            print('   🐢 任务2速度评估: 较慢')

    if total_times:
        avg_total = sum(total_times) / len(total_times)
        formatted_total_times = [f"{t:.1f}s" for t in total_times]
        print(f'\n⏱️ 双任务总体时间分析:')
        print(f'   平均总完成时间: {avg_total:.1f}秒')
        print(f'   最快总完成时间: {min(total_times):.1f}秒')
        print(f'   最慢总完成时间: {max(total_times):.1f}秒')
        print(f'   所有总完成时间: {formatted_total_times}')

    # 失败分析
    if failed > 0:
        timeout_rate = (timeout / failed * 100) if failed > 0 else 0
        print(f'\n❌ 失败详细分析:')
        print(f'   总失败率: {(failed / total * 100):.2f}%')
        print(f'   超时导致的失败: {timeout} 轮 ({timeout_rate:.1f}% of failures)')
        print(f'   其他原因失败: {failed - timeout} 轮')

        if task1_failed > 0:
            print(f'   任务1失败影响: {task1_failed} 轮测试因任务1失败而终止')
        if task2_failed > 0:
            print(f'   任务2失败影响: {task2_failed} 轮测试在任务2阶段失败')
        if task2_attempts < total:
            print(f'   任务2未尝试影响: {total - task2_attempts} 轮测试在任务1阶段终止')

    # 综合性能评估
    print(f'\n📈 综合性能评估:')
    if success_rate >= 80:
        print('   🟢 优秀 - 双任务模型表现非常好')
    elif success_rate >= 60:
        print('   🟡 良好 - 双任务模型表现较好，有改进空间')
    elif success_rate >= 40:
        print('   🟠 一般 - 双任务模型需要进一步优化')
    else:
        print('   🔴 较差 - 双任务模型需要重新训练或调整')

    # 针对性建议
    print(f'\n💡 优化建议:')
    if timeout > 0:
        print('   - 考虑优化模型推理速度或增加超时时间')
    if task1_failed > task2_failed:
        print('   - 任务1(移除红块)失败率较高，建议重点优化红块移除策略')
    elif task2_failed > task1_failed:
        print('   - 任务2(放置蓝杯)失败率较高，建议重点优化蓝杯放置策略')
    if success_rate < 70:
        print('   - 考虑收集更多双任务训练数据')
        print('   - 检查任务切换逻辑和policy配置')
        print('   - 验证环境设置和成功判断标准')
    if task1_times and task2_times:
        if avg_task1 > 60 or avg_task2 > 60:
            print('   - 考虑优化动作选择策略以提高执行效率')

    print('='*70)
    print('🎉 双任务测试完成！MuJoCo窗口已自动关闭')
else:
    print('没有双任务测试数据可供分析')


SmolVLA 双任务成功率测试 - 详细分析报告
📊 基本统计:
   总测试轮数: 20
   双任务成功: 4 轮
   测试失败: 16 轮
   超时失败: 16 轮

🎯 双任务成功率: 20.00%

🔍 任务失败分解分析:
   任务1(移除红块)成功率: 95.00% (19/20)
   任务2(放置蓝杯)成功率(全局): 20.00% (4/20)
   任务2(放置蓝杯)成功率(尝试): 21.05% (4/19)
   任务2未尝试: 1 轮 (因任务1失败)
   任务1失败: 1 轮
   任务2失败: 15 轮

⏱️ 任务1时间分析:
   平均完成时间: 29.9秒
   最快完成时间: 11.3秒
   最慢完成时间: 67.0秒
   所有完成时间: ['43.1s', '13.6s', '20.8s', '14.2s', '19.4s', '33.2s', '22.3s', '52.2s', '22.7s', '61.8s', '17.9s', '42.5s', '37.0s', '28.6s', '67.0s', '12.4s', '11.3s', '23.8s', '23.8s']
   ⚡ 任务1速度评估: 非常快速

⏱️ 任务2时间分析:
   平均完成时间: 12.7秒
   最快完成时间: 11.5秒
   最慢完成时间: 13.9秒
   所有完成时间: ['12.2s', '13.9s', '13.0s', '11.5s']
   ⚡ 任务2速度评估: 非常快速

⏱️ 双任务总体时间分析:
   平均总完成时间: 46.1秒
   最快总完成时间: 28.1秒
   最慢总完成时间: 67.8秒
   所有总完成时间: ['28.1s', '67.8s', '51.6s', '37.0s']

❌ 失败详细分析:
   总失败率: 80.00%
   超时导致的失败: 16 轮 (100.0% of failures)
   其他原因失败: 0 轮
   任务1失败影响: 1 轮测试因任务1失败而终止
   任务2失败影响: 15 轮测试在任务2阶段失败
   任务2未尝试影响: 1 轮测试在任务1阶段终止

📈 综合性能评估:
   🔴 较差 - 双任务模型需要重新训练或调整

💡 优化建议:
   

In [12]:
# [可选] 手动关闭MuJoCo窗口
# 如果需要在测试过程中手动关闭窗口，可以运行这个cell

# 取消注释下面的代码来手动关闭窗口
"""
if hasattr(PnPEnv, 'close_viewer'):
    PnPEnv.close_viewer()
    print("MuJoCo窗口已手动关闭")
else:
    print("环境不支持关闭窗口功能")
"""

print("手动关闭窗口功能已准备就绪（当前被注释）")


手动关闭窗口功能已准备就绪（当前被注释）


In [13]:
# policy.push_to_hub(
#     repo_id='Jeongeun/omy_pnp_smolvla',
#     commit_message='Add trained policy for PnP task',
# )